# Models subpackage tutorial

The NeuralModel class is a generic class used to manage neural networks implemented with Keras. It offers methods to save, load, train and use for classification the neural networks.

Melusine provides two built-in Keras model : cnn_model and rnn_model based on the models used in-house at Maif. However the user is free to implement neural networks tailored for its needs.

## The dataset

The NeuralModel class can take as input either :
- a text input : a cleaned text, usually the cleaned body or the concatenation of the cleaned body and the cleaned header.
- a text input and a metadata input : the metadata input has to be dummified.

#### Text input 

In [1]:
import pandas as pd

df_emails_preprocessed = pd.read_csv('./data/emails_preprocessed.csv', encoding='utf-8', sep=';')
df_emails_preprocessed['clean_header'] = df_emails_preprocessed['clean_header'].astype(str)
df_emails_preprocessed['clean_body'] = df_emails_preprocessed['clean_body'].astype(str)

In [2]:
df_emails_preprocessed.columns

Index(['body', 'header', 'date', 'from', 'to', 'label', 'is_begin_by_transfer',
       'is_answer', 'is_transfer', 'structured_historic', 'structured_body',
       'last_body', 'clean_body', 'clean_header', 'tokens'],
      dtype='object')

The new clean_text column is the concatenation of the clean_header column and the clean_body column :

In [3]:
df_emails_preprocessed['clean_text'] = df_emails_preprocessed['clean_header'] + " " + df_emails_preprocessed['clean_body']



In [4]:
df_emails_preprocessed.clean_text[0]

'devis habitation je suis client chez vous pouvez vous m etablir un devis pour mon fils qui souhaite louer l appartement suivant : 25 rue du rueimaginaire flag_cp_ '

#### Metadata input

By default the metadata used are :
- the extension : gmail, outlook, wanadoo..
- the day of the week at which the email has been sent
- the hour at which the email has been sent
- the minute at which the email has been sent

In [5]:
df_meta = pd.read_csv('./data/metadata.csv', encoding='utf-8', sep=';')

In [6]:
df_meta.columns

Index(['extension__0', 'extension__1', 'extension__2', 'extension__3',
       'extension__4', 'extension__5', 'extension__6', 'extension__7',
       'extension__8', 'dayofweek__0', 'dayofweek__1', 'dayofweek__3',
       'dayofweek__4', 'hour__6', 'hour__8', 'hour__9', 'hour__10', 'hour__11',
       'hour__12', 'hour__14', 'hour__15', 'hour__16', 'hour__17', 'hour__18',
       'hour__20', 'hour__22', 'min__0', 'min__2', 'min__4', 'min__6',
       'min__9', 'min__10', 'min__11', 'min__12', 'min__15', 'min__16',
       'min__19', 'min__20', 'min__21', 'min__22', 'min__24', 'min__28',
       'min__29', 'min__30', 'min__32', 'min__33', 'min__37', 'min__38',
       'min__39', 'min__40', 'min__44', 'min__45', 'min__49', 'min__54',
       'min__56', 'min__58'],
      dtype='object')

In [7]:
df_meta.head()

,extension__0,extension__1,extension__2,extension__3,extension__4,extension__5,extension__6,extension__7,extension__8,dayofweek__0,...,min__37,min__38,min__39,min__40,min__44,min__45,min__49,min__54,min__56,min__58
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Defining X and y

X is a Pandas dataframe with a clean_text column that will be used for the text input and columns containing the dummified metadata.

In [8]:
X = pd.concat([df_emails_preprocessed['clean_text'],df_meta],axis=1)

y is a numpy array containing the encoded labels :

In [9]:
from sklearn.preprocessing import LabelEncoder
y = df_emails_preprocessed['label']
le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
y

array([ 4, 10,  3,  0,  0,  4,  7, 10,  1, 10,  2,  5, 10, 10,  4,  7,  7,
       10,  0,  9,  4, 10,  4,  7, 10, 10,  6,  7,  3,  8, 10, 10, 10,  4,
        7,  3,  5,  4,  4, 10])

## The NeuralModel class

In [11]:
from melusine.models.train import NeuralModel

Using TensorFlow backend.


The NeuralModel class is a generic class used to manage neural networks implemented with Keras. It offers methods to save, load, train and use for classification the neural networks.

Its arguments are :
- **architecture_function :** a function returning a Model instance from Keras.
- **pretrained_embedding :** the pretrained embedding matrix as an numpy array.
- **text_input_column :** the name of the column that will provide the text input, by default clean_text.
- **meta_input_list :** the list of the names of the columns containing the metadata. If empty list or None the model is used without metadata. Default value, ['extension', 'dayofweek', 'hour', 'min'].
- **vocab_size :** the size of vocabulary for neurol network model. Default value, 25000.
- **seq_size :** the maximum size of input for neural model. Default value, 100.
- **loss :** the loss function for training. Default value, 'categorical_crossentropy'.
- **batch_size :** the size of batches for the training of the neural network model. Default value, 4096.
- **n_epochs :** the number of epochs for the training of the neural network model. Default value, 15.

#### architecture_function

In [12]:
from melusine.models.neural_architectures import cnn_model, rnn_model

**architecture_function** is a function returning a Model instance from Keras.
Melusine provides two built-in neural networks : **cnn_model** and **rnn_model** based on the models used in-house at Maif.

#### pretrained_embedding

The embedding have to be trained on the user's dataset.

In [13]:
from melusine.nlp_tools.embedding import Embedding

In [14]:
pretrained_embedding = Embedding().load('./data/embedding.pickle') 

### NeuralModel used with text and metadata input

This neural network model will use the **clean_text** column for the text input and the dummified **extension**, **dayofweek**, **hour** and **min** as metadata input :

In [15]:
nn_model = NeuralModel(architecture_function=cnn_model,
                       pretrained_embedding=pretrained_embedding,
                       text_input_column="clean_text",
                       meta_input_list=['extension', 'dayofweek', 'hour', 'min'],
                       n_epochs=10)

#### Training the neural network

In [16]:
nn_model.fit(X,y)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
40/40 [==============================] - 3s 78ms/step - loss: 2.4154 - acc: 0.1000
Epoch 2/10
40/40 [==============================] - 0s 10ms/step - loss: 2.3753 - acc: 0.1500
Epoch 3/10
40/40 [==============================] - 1s 14ms/step - loss: 2.3193 - acc: 0.3000
Epoch 4/10
40/40 [==============================] - 0s 12ms/step - loss: 2.2389 - acc: 0.3000
Epoch 5/10
40/40 [==============================] - 0s 8ms/step - loss: 2.0717 - acc: 0.3750
Epoch 6/10
40/40 [==============================] - 0s 7ms/step - loss: 1.9190 - acc: 0.3750
Epoch 7/10
40/40 [==============================] - 0s 7ms/step - loss: 1.9736 - acc: 0.3750
Epoch 8/10
40/40 [==============

#### Saving the neural network

The **save_nn_model** method saves :
- the Keras model as a json file 
- the weights as a h5 file

In [17]:
nn_model.save_nn_model("./data/nn_model")

Once the **save_nn_model** used the NeuralModel object can be saved as a pickle file :

In [18]:
from sklearn.externals import joblib
_ = joblib.dump(nn_model,"./data/nn_model.pickle",compress=True)

#### Loading the neural network

The NeuralModel saved as a pickle file has to be loaded first : 

In [19]:
nn_model = joblib.load("./data/nn_model.pickle")

Then the Keras model and its weights can be loaded :

In [20]:
nn_model.load_nn_model("./data/nn_model")

#### Making predictions 

In [21]:
y_res = nn_model.predict(X)
y_res = le.inverse_transform(y_res)

In [22]:
y_res

array(['habitation', 'vehicule', 'habitation', 'resiliation', 'vehicule',
       'habitation', 'resiliation', 'vehicule', 'habitation', 'vehicule',
       'habitation', 'habitation', 'vehicule', 'vehicule', 'habitation',
       'resiliation', 'resiliation', 'vehicule', 'vehicule', 'habitation',
       'habitation', 'vehicule', 'habitation', 'resiliation', 'vehicule',
       'vehicule', 'habitation', 'resiliation', 'vehicule', 'resiliation',
       'vehicule', 'vehicule', 'vehicule', 'habitation', 'resiliation',
       'vehicule', 'habitation', 'habitation', 'habitation', 'vehicule'],
      dtype=object)

### NeuralModel used with only text input

In [23]:
X = df_emails_preprocessed[['clean_text']]

In [24]:
nn_model = NeuralModel(architecture_function=cnn_model,
                       pretrained_embedding=pretrained_embedding,
                       text_input_column="clean_text",
                       meta_input_list=None,
                       n_epochs=10)

In [25]:
nn_model.fit(X,y)

Epoch 1/10
40/40 [==============================] - 2s 51ms/step - loss: 2.3995 - acc: 0.0250
Epoch 2/10
40/40 [==============================] - 0s 7ms/step - loss: 2.3798 - acc: 0.1000
Epoch 3/10
40/40 [==============================] - 0s 6ms/step - loss: 2.3203 - acc: 0.1250
Epoch 4/10
40/40 [==============================] - 0s 6ms/step - loss: 2.1830 - acc: 0.3000
Epoch 5/10
40/40 [==============================] - 0s 6ms/step - loss: 2.0731 - acc: 0.3750
Epoch 6/10
40/40 [==============================] - 0s 6ms/step - loss: 2.0091 - acc: 0.2250
Epoch 7/10
40/40 [==============================] - 0s 6ms/step - loss: 1.9957 - acc: 0.2750
Epoch 8/10
40/40 [==============================] - 0s 7ms/step - loss: 1.8969 - acc: 0.3500
Epoch 9/10
40/40 [==============================] - 0s 6ms/step - loss: 1.7454 - acc: 0.5250
Epoch 10/10
40/40 [==============================] - 0s 6ms/step - loss: 1.6435 - acc: 0.5000


In [26]:
y_res = nn_model.predict(X)
y_res = le.inverse_transform(y_res)
y_res

array(['habitation', 'vehicule', 'resiliation', 'resiliation',
       'resiliation', 'habitation', 'resiliation', 'vehicule',
       'resiliation', 'vehicule', 'vehicule', 'resiliation', 'vehicule',
       'vehicule', 'habitation', 'resiliation', 'resiliation', 'vehicule',
       'habitation', 'habitation', 'habitation', 'vehicule', 'habitation',
       'resiliation', 'vehicule', 'vehicule', 'vehicule', 'resiliation',
       'vehicule', 'resiliation', 'vehicule', 'vehicule', 'vehicule',
       'habitation', 'resiliation', 'vehicule', 'resiliation',
       'habitation', 'habitation', 'vehicule'], dtype=object)